In [1]:
import requests
import pandas as pd
import time

# 字段映射（API字段到中文）
mapping_field = {
    'f51': '日期',
    'f52': '开盘价',
    'f53': '收盘价',
    'f54': '最高价',
    'f55': '最低价',
    'f56': '成交量',
    'f57': '成交额',
    'f58': '振幅',
    'f59': '涨跌幅',
    'f60': '涨跌额'
}

class EastmoneyKlineFetcher:
    def __init__(self, stock_list, klt="101", fqt="1", lmt=120):
        self.stock_list = stock_list  # 格式如：['0.002040', '1.600519']
        self.klt = klt  # K线类型
        self.fqt = fqt  # 复权方式
        self.lmt = lmt  # 数据条数限制

    def fetch_single(self, secid):
        url = (
            f"https://push2his.eastmoney.com/api/qt/stock/kline/get?"
            f"secid={secid}&ut=fa5fd1943c7b386f172d6893dbfba10b"
            f"&fields1=f1,f2,f3,f4,f5,f6"
            f"&fields2={','.join(mapping_field.keys())}"
            f"&klt={self.klt}&fqt={self.fqt}&end=20500101&lmt={self.lmt}"
        )
        try:
            response = requests.get(url, timeout=10)
            json_data = response.json()
            if not json_data.get("data"):
                print(f"Warning: No data for {secid}")
                return None
            klines = json_data["data"]["klines"]
            df = pd.DataFrame(
                [line.split(',') for line in klines],
                columns=list(mapping_field.values())
            )
            df["股票代码"] = secid
            return df
        except Exception as e:
            print(f"Error fetching {secid}: {e}")
            return None

    def fetch_all(self):
        all_data = []
        for secid in self.stock_list:
            print(f"Fetching: {secid}")
            df = self.fetch_single(secid)
            if df is not None:
                all_data.append(df)
            time.sleep(0.5)  # 防止触发限速
        return pd.concat(all_data, ignore_index=True) if all_data else None

In [2]:
fetcher = EastmoneyKlineFetcher(['0.002040', '1.600519'], klt='101', fqt='1', lmt=90)
df = fetcher.fetch_all()
print(df.head())

Fetching: 0.002040
Fetching: 1.600519
           日期   开盘价   收盘价   最高价   最低价     成交量           成交额    振幅    涨跌幅  \
0  2024-12-03  7.28  7.47  7.57  7.17  279229  205548753.06  5.49   2.47   
1  2024-12-04  7.34  7.15  7.34  7.07  203252  146508447.53  3.61  -4.28   
2  2024-12-05  7.13  7.10  7.13  7.03  127005   89928929.17  1.40  -0.70   
3  2024-12-06  7.08  7.19  7.21  7.04  139553   99492299.17  2.39   1.27   
4  2024-12-09  7.14  7.18  7.28  7.05  131309   94041812.37  3.20  -0.14   

     涨跌额      股票代码  
0   0.18  0.002040  
1  -0.32  0.002040  
2  -0.05  0.002040  
3   0.09  0.002040  
4  -0.01  0.002040  
